# NYC Airbnb Price Prediction - Data Cleanup - HELP VERSION

Use dataset published by Kaggle - https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data - to train a simple deep learning model to predict prices for Airbnb properties.


This notebook contains the data cleanup steps:
- load data from the input CSV or dataframe saved by the [data preparation notebook](https://github.com/ryanmark1867/end_to_end_deep_learning_liveproject/blob/master/notebooks/data_preparation.ipynb)
- fix missing values
- clean up anomalies

# Common imports and variables
Imports and variable definitions that are common to the entire notebook


In [1]:
# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import matplotlib.pyplot as plt
# import datetime as dt
# # common imports
# import zipfile
# import time
# # import datetime, timedelta
# import datetime
# from datetime import datetime, timedelta
# from datetime import date
# from dateutil import relativedelta
# from io import StringIO
# import pandas as pd
# import pickle
# from sklearn.base import BaseEstimator
# from sklearn.base import TransformerMixin
# from io import StringIO
# import requests
# import json
# from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
# from sklearn.model_selection import train_test_split
# import matplotlib.pyplot as plt
# %matplotlib inline 
# import os
# import math
# from subprocess import check_output
# from IPython.display import display
# import logging
# import yaml
# from collections import Counter
# import re
# import os
# import numbers
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import logging
import yaml
from collections import Counter
import os

In [2]:
def get_config(config_file):
    ''' open config file with name config_file that contains parameters
    for this module and return Python object

    Args:
        config_file: filename containing config parameters

    Returns:
        config: Python dictionary with config parms from config file - dictionary


    '''
    current_path = os.getcwd()
    path_to_yaml = os.path.join(current_path, config_file)
    print("path_to_yaml " + path_to_yaml)
    try:
        with open(path_to_yaml, 'r') as c_file:
            config = yaml.safe_load(c_file)
        return config
    except Exception as error:
#       print('Error reading the config file ' + str(error))
        print('Error reading the config file ' + error)

In [3]:
def print_config_values(config):
    ''' print values in dictionary config'''
    for val in config:
        print(f"config value {val} = {config[val]}" )

# Load dataframe
- load pickled dataframe


In [4]:
def get_path():
    ''' get the path for data files

    Returns:
        path: path for data directory

    '''
    rawpath = os.getcwd()
    # data is in a directory called "data" that is a sibling to the directory
    # containing the notebook
    path = os.path.abspath(os.path.join(rawpath, '..', 'data'))
    return path

In [5]:
def ingest_data(path,input_csv,pickled_input_dataframe,save_raw_dataframe,load_from_scratch):
    ''' load data into dataframe
    Args:
        path: path containing input file
        input_csv: input file name
        pickled_input_dataframe: pickled version of input file

    Returns:
        path: path for data directory
    '''
    if load_from_scratch:
        # if loading from scratch, the raw CSV file is expected to be in the data directory which is a sibling to the 
        # directory that contains this notebook
        unpickled_df = pd.read_csv(os.path.join(path,input_csv)) 
        if save_raw_dataframe:
            file_name = os.path.join(path,pickled_input_dataframe)
            print("file_name is ",file_name)
            unpickled_df.to_pickle(file_name)
    else:
        unpickled_df = pd.read_pickle(os.path.join(path,pickled_input_dataframe))
        logging.debug("reloader done")
    return(unpickled_df)

# General cleanup


In [6]:
def fill_missing(dataset,columns,defaults):
    ''' replace missing values with placeholders by column type
    
    Args:
        dataset: dataframe in which missing values being processed
        columns: dictionary of columns with keys that are column types and values that are the column names of that type
        defaults: dictionary of replacement values for missing values by column type

    Returns:
        dataset: dataframe with missing values replaced with default values
         
    '''
    for ctype in columns: #.keys():
        for cname in columns[ctype]:
            print(f"# missing values BEFORE cleaning ds[{cname}] = {dataset[cname].isna().sum()}")
            dataset[cname].fillna(defaults[ctype], inplace=True)        
            print(f"# missing values AFTER cleaning ds[{cname}] = {dataset[cname].isna().sum()}")
    return dataset

In [7]:
def not_in_list(x, list):
    ''' check if a value is in a list
    Args:%ls ../../data
        x: value to check
        list: list in which to check for the value

    Returns:
        retur_val: 1 if value is in not in list, 0 otherwise
    '''
    return not x in list

In [8]:
def neg_val(x):
    ''' check if a value is in a list
    Args:
        x: value to check
    
    Returns:
        retur_val: 1 if value is negative, 0 otherwisnamee
    '''
    return x < 0

In [9]:
def basic_assessment(df,columns,valid_values,non_neg_continuous):
    ''' assess the values in a dataframe
    Args:
        df: dataframe for assessment
        columns: dictionary of column names by category
        valid_values: dictionary of valid values for categorical columns with limited number of valid values
        non_neg_continuous: list of continuous columns with only non-negative values as valid
    '''
# No need to wrap the df in a list to get the columns! But look out below! 
# columns is also the YML field passed in!  
    for col in df.columns:
        print("Missing values in ",col," ",df[col].isna().sum())
        print("Distinct values in ",col," ", df[col].nunique())

    # for categorical columns with a limited number of valid values, count the number of invalid values by column
    for col in valid_values:
         print("non-valid values in column ",col," ", df[col].apply(lambda x:not_in_list(x,valid_values[col])).sum())
    
    # count non-numeric values in continuous columns
    for col in columns['continuous']:
        mask = pd.to_numeric(df[col], errors='coerce').isna()
        # count non-numeric values in continuous columns
        print("non-numeric values in continuous col ",col," ", mask.sum())
        # if there are no non-numeric values in the column and it must have non-negative values, count negative values
        if (mask.sum()==0) and (col in non_neg_continuous):
            print("negative values in column ",col," ", df[col].apply(lambda x:neg_val(x)).sum())
    

In [10]:
def out_of_range(x,max,min):
    ''' count whether a value is in a range
    Args:
        x: value to check in range
        max: top of the range to check
        min: bottom of the range to check
        
    Returns:
        ret_val: 1 if out of range, 0 otherwise
    '''
    return x < min or x > max


In [11]:
def out_of_bounding_box(latitude,longitude,bounding_box):
    ''' count whether a location is within a bounding box
        NOTE: Also checks for invalid lat/longs, if any.
        TODO: Since lat/long are Series objects, might be 
        more optimal to combine them into a df and then
        have apply return a bunch of tuples?
    Args:
        latitude: latitude portion of location
        longitude: longitude portion of location
        bounding_box: dictionary with max and min values to compare the location with
        min: bottom of the range to check
        
    Returns:
        count of OOBB/invalid lats/longs
        #ret_val: 1 if out of range, 0 otherwise
    '''    
    min_lat =  bounding_box['min_lat']
    max_lat =  bounding_box['max_lat']
    min_long = bounding_box['min_long']
    max_long = bounding_box['max_long']  
    
    t1 = time.perf_counter_ns()
                            
    total = (
#         latitude[latitude > 90].sum() + #alternately
        sum(latitude > 90)
        + sum(latitude < -90) + sum(longitude > 180) 
        + sum (longitude < -180) + sum(latitude > max_lat) + sum(latitude < min_lat) 
        + sum(longitude > max_long) + sum(longitude < min_long)
    )   
    print(f"lat/long checks took {time.perf_counter_ns() - t1} ns.")   
    return total        
    

In [12]:
def geo_assessment(df,bounding_box):
    ''' assess the geo columns in a dataframe by counting how many latitude and longitude values are outside the bounding box
    Args:
        df: dataframe for assessment
        bounding_box: dictionary of maximum and minimum valid latitude and longitude values
    ''' 
    oobb = out_of_bounding_box(df.latitude,df.longitude,bounding_box)
    print("location out of bounds count ", oobb)

In [13]:
def replace_if_non_numeric(x, replace_x):
    ''' check if a value is non-numeric and replace if so
    Args:
        x: value to check
    
    Returns:
        retur_val: 1 if value is negative, 0 otherwise
    '''
    if pd.to_numeric(x, errors='coerce').isna():
        return replace_x
    return x


In [14]:
def replace_if_not_in_list(x, replace_x, list):
    ''' check if a value is in a list
    Args:
        x: value to check
        list: list in which to check for the value

    Returns:
        retur_val: 1 if value is in not in list, 0 otherwise
    '''
    if x not in list:
        return replace_x
    return x

In [15]:
def replace_if_neg(x, replace_x):
    ''' check if a value is negative and replace if so
    Args:
        x: value to check
    
    Returns:
        retur_val: 1 if value is negative, 0 otherwise
    '''
    if x < 0:
        return replace_x
    return x

In [16]:
def geo_replacement(df,replace_lat, replace_long, bounding_box):
    ''' assess the geo columns in a dataframe by counting how many latitude and longitude values are outside the bounding box
    Args:
        df: dataframe for assessment
        replace_lat: replacement for latitude if out of bounds
        replace_long: replacement for longitude if out of bounds
        bounding_box: dictionary of maximum and minimum valid latitude and longitude values
    Returns:
        df: updated dataframe
    ''' 
    min_lat =  bounding_box['min_lat']
    max_lat =  bounding_box['max_lat']
    min_long = bounding_box['min_long']
    max_long = bounding_box['max_long']
    df['latitude','longitude'] = df.apply(lambda x:replace_if_outside_bounding_box(x.latitude,x.longitude,replace_lat,replace_long,bounding_box),axis=1)

 

In [17]:
def replace_if_outside_bounding_box(latitude,longitude, replace_lat, replace_long, bounding_box):
    ''' count whether a location is within a bounding box
    Args:
        latitude: latitude portion of location
        longitude: longitude portion of location
        replace_lat: replacement value for latitude
        replace_long: replacement value for longitude
        bounding_box: dictionary with max and min values to compare the location with
               
    Returns:
        latitude, longitude: 1 if out of range, 0 otherwise
    ''' 
    min_lat =  bounding_box['min_lat']
    max_lat =  bounding_box['max_lat']
    min_long = bounding_box['min_long']
    max_long = bounding_box['max_long']
    lat = latitude
    if latitude < -90 or latitude > 90 or latitude > max_lat or latitude < min_lat:
        lat = replace_lat
    long = longitude    
    if longitude < -180 or  longitude > 180 or longitude > max_long or longitude < min_long:
        long = replace_long
    return lat, long
    

In [18]:
def replace_time(date_time_value,time_value):
    ''' given a datetime replace the time portion '''
    date_time_value = date_time_value.replace(hour=time_value.hour,minute=time_value.minute,second=time_value.minute)
    return date_time_value

In [19]:
def replace_invalid_values(df,columns,valid_values,invalid_value_replacements,non_neg_continuous):
    ''' replace invalid with placeholders
    Args:
        df: dataframe for assessment
        columns: dictionary of column names by category
        valid_values: dictionary of valid values for categorical columns with limited number of valid values
        invalid_value_replacements: dictionary of replacement values by column category
        non_neg_continuous: list of continuous columns with only non-negative values as valid
    Returns:
        df: updated dataframe
    '''
    repl_cat = invalid_value_replacements["categorical"]
    repl_real = invalid_value_replacements["continuous"]
    
    for col in columns["categorical"]:
        df[col] = df[col].apply(lambda x: replace_if_not_in_list(x, repl_cat, valid_values))  

    for col in columns["continuous"] + columns["excluded"]:
        if col in non_neg_continuous:
            df[col] = df[col].apply(lambda x: replace_if_neg(x, repl_real))

# Master cell
This cell contains calls to the other functions in this notebook to complete the data cleanup

In [20]:
# master cell to call the other functions
# get the path for data files
t1 = time.perf_counter_ns()
path = get_path()
print("path is ",path)
config = get_config('data_preparation_config.yml')
print("past config definition")
logging.getLogger().setLevel(logging.WARNING)
logging.warning("logging check")
print_config_values(config)

# Refactored
column_config = config['columns']
defaults_config = config['category_defaults']
filename_config = config['file_names']
validvalues_config = config['valid_values']
non_neg_config = config['non_negative_continuous']
invalid_value_config = config['category_invalid_value_replacements']

# load dataframe
df = ingest_data(path,filename_config['input_csv'],filename_config['pickled_input_dataframe'],config['general']['save_raw_dataframe'],config['general']['load_from_scratch'])

# TODO! ONLY for test!!!!
#df = df.sample(n=1000, random_state=42)

# print("columns is "+str(config['columns']))
# print("category_defaults is "+str(config['category_defaults']))
print(f"columns are {column_config}.")
print(f"category_defaults is {defaults_config}")
# fill missing values according to the defaults per column
fill_missing(df, column_config, defaults_config)

# get assessment results after filling missing values
basic_assessment(df, column_config, validvalues_config, non_neg_config)
# df = replace_invalid_values(df,columns,valid_values,invalid_value_replacements,non_neg_continuous)
#df = 
replace_invalid_values(df,column_config, validvalues_config, invalid_value_config, non_neg_config)
geo_assessment(df,config['bounding_box'])
'''
df = geo_replacement(df,config['latitude_replacement'],config['longitude_replacement'], config['bounding_box'])
'''
if config['general']['save_transformed_dataframe']:
    print("path is ",path)    
    file_name = os.path.join(path,config['file_names']['pickled_output_dataframe'])
    print("file_name is ",file_name)
    df.to_pickle(file_name)
df.head()
print(f"All done  in {time.perf_counter_ns() - t1} ns!!!")

path is  /media/srutis/Acer/Users/SRUTIS/projects/Manning/DLAirBnBPrices-LP/project1-prep-tab-data/data
path_to_yaml /media/srutis/Acer/Users/SRUTIS/projects/Manning/DLAirBnBPrices-LP/project1-prep-tab-data/ms2/data_preparation_config.yml
past config definition
config value general = {'load_from_scratch': True, 'save_raw_dataframe': True, 'save_transformed_dataframe': True, 'remove_bad_values': True}
config value columns = {'categorical': ['neighbourhood_group', 'neighbourhood', 'room_type'], 'continuous': ['minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'latitude', 'longitude'], 'date': ['last_review'], 'text': ['name', 'host_name'], 'excluded': ['price', 'id']}
config value category_defaults = {'categorical': 'missing', 'continuous': 0.0, 'text': 'missing', 'date': datetime.date(2019, 1, 1), 'excluded': 'missing'}
config value category_invalid_value_replacements = {'categorical': 'bad_categorical', 'continuous': 'bad_continuous', 'text': 